In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch
import torchvision.transforms as T
from torchvision.utils import make_grid
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from pathlib import Path

%matplotlib inline

In [2]:
train = Path(r'C:\Users\USER-PC\Documents\Data Science\data set\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train')
val =Path(r'C:\Users\USER-PC\Documents\Data Science\data set\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid')

In [3]:
from torchvision.datasets import ImageFolder

In [4]:
image_size = 32
stats = ((0.5, 0.45, 0.425), (0.35, 0.5, 0.253))

dataset = ImageFolder(train, transform = T.Compose([T.CenterCrop(image_size), T.Resize(image_size),
                                                         T.ToTensor(), T.Normalize(*stats)]))
validation_set = ImageFolder(val, transform = T.Compose([T.CenterCrop(image_size), T.Resize(image_size),
                                                        T.ToTensor(), T.Normalize(*stats)]))

In [5]:
dataset = ImageFolder(train, transform = T.Compose([T.Resize(image_size), T.ToTensor()]))
validation_set = ImageFolder(val, transform = T.Compose([T.Resize(image_size), T.ToTensor()]))

In [7]:
print(len(dataset.classes))
dataset.classes

38


['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [ ]:
img, label = dataset[0]
print("label", dataset.classes[label], ':', label)
print(img.shape)
plt.imshow(img.permute(1, 2, 0))

In [ ]:
random_seed = 20
torch.manual_seed(random_seed)
batch_size = 250

In [ ]:
len(dataset), len(validation_set)

In [ ]:
train_loader = DataLoader(dataset, batch_size, shuffle= True, collate_fn=None, num_workers=4, pin_memory = True)
val_loader = DataLoader(validation_set, batch_size=batch_size*2, num_workers=4, collate_fn=None, pin_memory=True)

In [ ]:
def showbatch(data):
    for image, label in data:
        fig, ax = plt.subplots(figsize = (16, 8))
        plt.xticks([]); plt.yticks([])
        plt.imshow(make_grid(image, nrow=16).permute(1, 2, 0))
        break

In [ ]:
showbatch(train_loader)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim =1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        acc = accuracy(out, labels)
        loss = F.cross_entropy(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        batch_accuracies = [x['val_acc'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        epoch_accuracy = torch.stack(batch_accuracies).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_accuracy.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch[{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        for batch in train_loader:
            loss = model.training_step(batch)
            loss = loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
        return history

In [ ]:
torch.cuda.is_available()

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [ ]:
device = get_default_device()
device

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
def plot_losses(history):
    losses = [x['val_loss'] for x in history]
    plt.plot(losses, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title("Loss Vs. No of epoochs")

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title("Accuracy Vs. No of epochs")

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

In [ ]:
input_size = 3*256*256
hidden_size1 = 1024
hidden_size2 = 512 
hidden_size3 = 256
hidden_size4 = 128
hidden_size5 = 8
output_size = 38
batch_size = 190

In [ ]:
class PlantDiseaseModel(ImageClassificationBase):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size3)
        self.linear4 = nn.Linear(hidden_size3, hidden_size4)
        self.linear5 = nn.Linear(hidden_size4, output_size)
        
        
    def forward(self, xb):
        out = xb.view(xb.size(0), -1)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        out = F.relu(out)
        out = self.linear3(out)
        out = F.relu(out)
        out = self.linear4(out)
        out = F.relu(out)
        out = self.linear5(out)
        return out

In [ ]:
model = to_device(PlantDiseaseModel(input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, output_size), device)

In [ ]:
history = [evaluate(model, val_loader)]
history

In [ ]:
history += fit(10, 0.1, model, train_loader, val_loader)

In [ ]:
plot_losses(history)

In [ ]:
plot_accuracies(history)

In [ ]:
def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds = torch.max(yb, dim = 1)
    return dataset.classes[preds[0].item()]

In [ ]:
img, label = train[100]
plt.imshow(img.permute(1, 2, 0))
print("Label: ", dataset.classes[label], ", Predicted: ", predict_image(img, model))

In [ ]:
img, label = dataset[38000]
plt.imshow(img.permute(1, 2, 0))
print("Label", dataset.classes[label], ", Predicted", predict_image(img, model))

In [ ]:
val_loader = DataLoader(val, batch_size*3, shuffle = True, num_workers = 3, collate_fn = None, pin_memory = True)
result = evaluate(model, val_loader)
result